In [1]:
import numpy as np
from scipy.optimize import linprog
import networkx as nx
import matplotlib

In [43]:
def draw_graph(G):
    nx_G = nx.from_dict_of_lists(G)
    nx.draw_networkx(nx_G)

# Linear Program for fractional clique number

Define a function to return all possible subsests for list of vertices V

In [3]:
def subsets(V, prefix=[], arr_sets=[]):
    if V == []: return
    for i in range(len(V)):
        subset = prefix + [V[i]]
        arr_sets += [subset]
        subsets(V[i+1:], subset, arr_sets)

In [4]:
def check_clique(graph, v_set):
    if len(v_set) < 2: return True
    for i in range(len(v_set)):
        i_set = set(v_set[:i] + v_set[i+1:])
        g_set = graph[v_set[i]]
        if i_set & g_set != i_set :
            return False
    return True

In [5]:
def A_clique_set(graph, v_encode, subsets, A=[]):
    for subset in subsets:
        if check_clique(graph, subset):
            A += [sum([v_encode[v] for v in subset])]

In [6]:
def lp_clique(graph):
    V = list(graph.keys())
    v_encode = {}
    v_num = len(V)
    
    for i in range(v_num):
        temp = np.zeros(v_num)
        temp[i] = -1
        v_encode[V[i]] = temp
    
    arr_sets = []
    subsets(V, arr_sets=arr_sets)
    
    b_clique = np.negative(np.ones(v_num))
    A_clique = []
    A_clique_set(graph, v_encode, arr_sets,A=A_clique)
    A_clique = np.transpose(np.array(A_clique))
    c_clique = np.ones(A_clique.shape[1])
    
    res = linprog(c_clique, A_ub=A_clique, b_ub=b_clique, bounds=(0, None))
    return res, A_clique

# Linear Program for Shannon entropy

In [7]:
def remove_unconnected_v(graph):
    keys = list(graph.keys())
    for v in keys:
        if len(graph[v]) == 0:
            graph.pop(v, None)

In [8]:
G10 = {1:set({}), 2:set({}), 3:set({}), 4:set({}), 5:set({}), 6:set({})}

In [9]:
remove_unconnected_v(G10)

In [10]:
G10

{}

In [11]:
def subsets_dict(V, prefix=[], arr_sets={}):
    if V == []: return 1
    for i in range(len(V)):
        subset = prefix + [V[i]]
        arr_sets[tuple(subset)] = len(arr_sets)
        subsets_dict(V[i+1:], subset, arr_sets)

In [12]:
def A_entropy_set(V, subsets_dict, A=[], b=[]):
    num_sets = len(subsets_dict)
    for subset in subsets_dict:
        subset_len = len(subset)
        subset_i = subsets_dict[subset]
        a_lq = np.zeros(num_sets)
        if subset_len > 1:
            a_lq[subset_i] = -1 * (subset_len)
            a_mq = np.zeros(num_sets)
            a_mq[subset_i] = subset_len - 1
            s = set(subset)
            for v in subset:
                sub_i = subsets_dict[tuple(x for x in subset if v != x)]
                a_lq[sub_i] = 1    
                a_mq[sub_i] = -1
            A += [a_lq, a_mq]
            b += [0, 0]
        else:
            a_lq[subset_i] = 1
            A += [a_lq]
            b += [1]

In [13]:
def A_entropy_set_2(V, subsets_dict, A=[], b=[]):
    num_sets = len(subsets_dict)
    subsets = list(subsets_dict.keys())
    for i in range(num_sets):
        subset_i = subsets[i]
        pos_i = subsets_dict[subset_i]
        for j in range(i, num_sets):
            subset_j = subsets[j]
            pos_j = subsets_dict[subset_j]
            a_lq = np.zeros(num_sets)
            if len(subset_i) < 2 and pos_i == pos_j: 
                a_lq[pos_i] = 1
                A += [a_lq]
                b += [1]
#                 print("identical", subset_i)
            else:
                a_mq = np.zeros(num_sets)
                subset_i = set(subset_i)
                subset_j = set(subset_j)
                subset_ij_u = subset_i | subset_j
                subset_ij_i = subset_i & subset_j
                if (subset_ij_i == subset_i or subset_ij_i == subset_j):
                    if len(subset_i) > len(subset_j):
                        a_mq[pos_i] = -1
                        a_mq[pos_j] = 1
                        A += [a_mq]
                        b += [0]
                    elif len(subset_i) < len(subset_j):
                        a_mq[pos_i] = 1
                        a_mq[pos_j] = -1
                        A += [a_mq]
                        b += [0]
#                     print("intersect each other", subset_i, subset_j)
                else:
#                     print("not intersection of each other", subset_i, subset_j)
                    a_lq[pos_i] = -1
                    a_lq[pos_j] = -1
                    a_lq[subsets_dict[tuple(subset_ij_u)]] = 1
                    if len(subset_ij_i) > 0:
                        a_lq[subsets_dict[tuple(subset_ij_i)]] = 1
                    A += [a_lq]
                    b += [0]

This version is missing a lot of important conditions

In [14]:
def A_entropy_set_3(V, subsets_dict, A=[], b=[]):
    num_sets = len(subsets_dict)
    subsets = list(subsets_dict.keys())
    for i in range(num_sets):
        subset_i = subsets[i]
        pos_i = subsets_dict[subset_i]
        for j in range(i, num_sets):
            subset_j = subsets[j]
            pos_j = subsets_dict[subset_j]
            a_mq = np.zeros(num_sets)
            if len(subset_i) < 2 and pos_i == pos_j: 
                a_mq[pos_i] = 1
                A += [a_mq]
                b += [1]
            else:
                subset_i = set(subset_i)
                subset_j = set(subset_j)
                subset_ij_u = subset_i | subset_j
                subset_ij_i = subset_i & subset_j
                if (subset_ij_i == subset_i or subset_ij_i == subset_j):
                    if len(subset_i) > len(subset_j):
                        a_mq[pos_i] = -1
                        a_mq[pos_j] = 1
                        A += [a_mq]
                        b += [0]
                    elif len(subset_i) < len(subset_j):
                        a_mq[pos_i] = 1
                        a_mq[pos_j] = -1
                        A += [a_mq]
                        b += [0]
        
        if len(subset_i) > 1:
            a_lq = np.zeros(num_sets)
            a_lq[pos_i] = 1
            for v in subset_i:
                sub_i = subsets_dict[(v,)]
                a_lq[sub_i] = -1
            A += [a_lq]
            b += [0]


In [15]:
def entropy_eq_dict(graph, eq_dict={}):
    for v in graph:
        n = tuple(graph[v])
        n_v = tuple(graph[v] | {v})
        if n in eq_dict:
            eq_dict[n_v] = eq_dict[n]
        else:
            eq_dict[n] = n_v    

In [134]:
def A_entropy_set_4(graph, subsets_dict, eq_dict,  A=[]):
    num_sets = len(subsets_dict)
    subsets = list(subsets_dict.keys())
    for i in range(num_sets):
        subset_i = subsets[i]
        pos_i = subsets_dict[subset_i]
        rep_i = subset_i in eq_dict
        for j in range(i + 1, num_sets):
            subset_j = subsets[j]
            pos_j = subsets_dict[subset_j]
            rep_j = subset_j in eq_dict            
            
            val_i, val_j = (), ()
            
            if rep_i:
                pos_i = subsets_dict[eq_dict[subset_i]]
            if rep_j:
                pos_j = subsets_dict[eq_dict[subset_j]]

            subset_i_s = set(subset_i)
            subset_j_s = set(subset_j)
            subset_ij_u = tuple(subset_i_s | subset_j_s)
            subset_ij_i = tuple(subset_i_s & subset_j_s)
            
            pos_u = subsets_dict[subset_ij_u]
            
            if subset_ij_u in eq_dict:
                pos_u = subsets_dict[eq_dict[subset_ij_u]]
            
            if (subset_ij_i == subset_i or subset_ij_i == subset_j):
                if pos_i == pos_j: 
#                     print('skipped mq same pos', subset_i, subset_j, subset_ij_u, subset_ij_i, pos_i, pos_j)
                    continue
                diff = len(subset_i) - len(subset_j)
                a_mq = np.zeros(num_sets) 
                if diff == 1:
                    a_mq[pos_i] = -1
                    a_mq[pos_j] = 1
                    A += [a_mq]
#                     print('mq', subset_i, subset_j, subset_ij_u, subset_ij_i, pos_i, pos_j, a_mq)
                elif diff == -1:
                    a_mq[pos_i] = 1
                    a_mq[pos_j] = -1
                    A += [a_mq]
#                     print('mq', subset_i, subset_j, subset_ij_u, subset_ij_i, pos_i, pos_j, a_mq)
            else:
                if ((pos_i == pos_u and pos_j != pos_u) or 
                (pos_j == pos_u and pos_i != pos_u)): 
#                     print('skipped lq', subset_i, subset_j, subset_ij_u, subset_ij_i, pos_i, pos_j)
                    continue
                a_lq = np.zeros(num_sets) 
                
                a_lq[pos_i] = -1
                a_lq[pos_j] = -1
                a_lq[pos_u] = 1
                
                if len(subset_ij_i) > 0:
                    if subset_ij_i in eq_dict:
                        pos_ij = subsets_dict[eq_dict[subset_ij_i]]
                        if pos_i != pos_j:
                            if pos_i == pos_ij:
                                a_lq[pos_i] = 0
                            elif pos_j == pos_ij:
                                a_lq[pos_j] = 0
                            else:
                                a_lq[pos_ij] = 1
                    else:
                        pos_ij = subsets_dict[subset_ij_i]
                        a_lq[pos_ij] = 1
                A += [a_lq]
#                 print('lq', subset_i, subset_j, subset_ij_u, subset_ij_i, pos_i, pos_j, a_lq)

In [17]:
def A_entropy_eq_set(graph, subsets_dict, A=[]):
    num_sets = len(subsets_dict)
    for v in graph:
        n = tuple(graph[v])
        n_v = tuple(graph[v] | {v})
        a_eq = np.zeros(num_sets)
        a_eq[subsets_dict[n_v]] = 1
        if len(n) > 0: 
            a_eq[subsets_dict[n]] = -1
        A += [a_eq]

In [18]:
def A_entropy_eq_set_2(subsets_dict, eq_dict, A=[]):
    num_sets = len(subsets_dict)
    for k in eq_dict:
        v = eq_dict[k]
        a_eq = np.zeros(num_sets)
        a_eq[subsets_dict[k]] = 1
        a_eq[subsets_dict[v]] = -1
        A += [a_eq]

In [63]:
def remove_duplicate_A(A=np.array([])):
    temp_a = np.ascontiguousarray(A)
    unique_a = np.unique(temp_a.view([('', temp_a.dtype)]*temp_a.shape[1]))
    A = unique_a.view(temp_a.dtype).reshape((unique_a.shape[0], temp_a.shape[1]))
    return A

In [65]:
def add_v_Ab(V, subsets_dict, A=np.array([]), b=np.array([])):
    A = list(A)
    b = b.tolist()
    print(len(A))
    num_sets = len(subsets_dict)
    for _v in V:
        pos_v = subsets_dict[(_v,)]
        temp_a = np.zeros(num_sets)
        temp_a[pos_v] = 1
        A += [temp_a]
        b += [1]
    return np.array(A), np.array(b)

In [120]:
def lp_entropy_1(graph):
    # remove all isolated vertices from the graph
    remove_unconnected_v(graph)
        
    V = list(graph.keys())
    v_num = len(V)
    
    if v_num == 0: return 0
    
    arr_sets = {}
    subsets_dict(V, arr_sets=arr_sets)
    
    eq_dict = {}
    entropy_eq_dict(graph, eq_dict=eq_dict)
    
    A_entropy = []
    b_entropy = []
    A_entropy_set_2(graph, arr_sets, A=A_entropy, b=b_entropy)
    A_entropy = np.array(A_entropy)
    b_entropy = np.array(b_entropy, dtype=float)
    
    A_entropy_eq = []
    A_entropy_eq_set(graph, arr_sets, A=A_entropy_eq)
    A_entropy_eq = np.array(A_entropy_eq)
    b_entropy_eq = np.zeros(A_entropy_eq.shape[0])
    
    c_clique = np.zeros(len(arr_sets))
    c_clique[arr_sets[tuple(V)]] = -1
    
    print(A_entropy, A_entropy.shape)
    
    res = linprog(c_clique, A_ub=A_entropy, b_ub=b_entropy, A_eq=A_entropy_eq, b_eq=b_entropy_eq, bounds=(0, None))
    
    x = res.x
    keys = list(arr_sets.keys())
    for i in range(x.shape[0]):
        print(keys[i], x[i])
    return res

In [66]:
def lp_entropy(graph):
    # remove all isolated vertices from the graph
    remove_unconnected_v(graph)
        
    V = list(graph.keys())
    v_num = len(V)
    
    if v_num == 0: return 0
    
    arr_sets = {}
    subsets_dict(V, arr_sets=arr_sets)
    
    eq_dict = {}
    entropy_eq_dict(graph, eq_dict=eq_dict)
    
    print(eq_dict)
    
    A_entropy = []
    A_entropy_set_4(graph, arr_sets, eq_dict, A=A_entropy)
    A_entropy = np.array(A_entropy)
    print(A_entropy, A_entropy.shape)
    
    A_entropy = remove_duplicate_A(A=A_entropy)
    b_entropy = np.zeros(A_entropy.shape[0])
    
    print(A_entropy, A_entropy.shape)
    A_entropy, b_entropy = add_v_Ab(V, arr_sets, A=A_entropy, b=b_entropy)
    
    A_entropy_eq = []
    A_entropy_eq_set_2(arr_sets, eq_dict, A=A_entropy_eq)
    A_entropy_eq = np.array(A_entropy_eq)
    b_entropy_eq = np.zeros(A_entropy_eq.shape[0])
    
    c_clique = np.zeros(len(arr_sets))
    c_clique[arr_sets[tuple(V)]] = -1
    
    print(A_entropy, A_entropy.shape)
    
    res = linprog(c_clique, A_ub=A_entropy, b_ub=b_entropy, A_eq=A_entropy_eq, b_eq=b_entropy_eq, bounds=(0, None))
    
    x = res.x
    keys = list(arr_sets.keys())
    for i in range(x.shape[0]):
        print(keys[i], x[i])
    return res

# Sample Graphs

In [136]:
G1 = {1:{2,5,6}, 2:{1,3,6}, 3:{2,4}, 4:{3,5,7}, 5:{1,4}, 6:{1,2,7}, 7:{4,6}}
G2 = {1:{2,5,6}, 2:{1,3,7}, 3:{2,4,6}, 4:{3,5}, 5:{1,4}, 6:{1,3,7}, 7:{2,6}}
G3 = {1:{2,5,6}, 2:{1,3}, 3:{2,4,6}, 4:{3,5,7}, 5:{1,4}, 6:{1,3,7}, 7:{4,6}}
G4 = {1:{2,5,6}, 2:{1,3,7}, 3:{2,4,6}, 4:{3,5,7}, 5:{1,4}, 6:{1,3,7}, 7:{2,4,6}}
G5 = {1:{2,5,6}, 2:{1,3,7}, 3:{2,4}, 4:{3,5}, 5:{1,4}, 6:{1,7}, 7:{2,6}}
G6 = {1:{2,5,6}, 2:{1,3}, 3:{2,4,7}, 4:{3,5}, 5:{1,4}, 6:{1,7}, 7:{3,6}}
G7 = {1:{2,3,4,5,6,7}, 2:{1,3,4,5,6,7}, 3:{1,2,4,5,6,7}, 4:{1,2,3,5,6,7}, 5:{1,2,3,4,6,7}, 6:{1,2,3,4,5,7}, 7:{1,2,3,4,5,6}}

In [116]:
G8 = {1:{2,5}, 2:{1,3}, 3:{2,4}, 4:{3,5}, 5:{1,4}}

In [33]:
G9 = {1:{2,6}, 2:{1,3}, 3:{2,4}, 4:{3,5}, 5:{4,6}, 6:{1,5}}
G10 = {1:{2}, 2:{1}, 3:set({}), 4:set({}), 5:set({}), 6:set({})}

In [46]:
G11 = {1:{3,4}, 2:{3,4}, 3:{1,2}, 4: {1,2}}
draw_graph(G11)

In [ ]:
lp_entropy(G1)

{(2, 5, 6): (1, 2, 5, 6), (1, 3, 6): (1, 2, 3, 6), (2, 4): (2, 3, 4), (3, 5, 7): (3, 4, 5, 7), (1, 4): (1, 4, 5), (1, 2, 7): (1, 2, 6, 7), (4, 6): (4, 6, 7)}
[[ 1. -1.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ... -1.  1. -1.]
 [ 0.  0.  0. ...  0. -1.  1.]] (6466, 127)
[[-1.  0.  0. ...  0.  0. -1.]
 [-1.  0.  0. ...  0. -1.  0.]
 [-1.  0.  0. ... -1.  0.  0.]
 ...
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 1.  0.  0. ...  0.  0.  0.]] (6466, 127)
6466
[[-1.  0.  0. ...  0.  0. -1.]
 [-1.  0.  0. ...  0. -1.  0.]
 [-1.  0.  0. ... -1.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  1.]] (6473, 127)


In [ ]:
G11 = {1:{2,3,4,5}, 2:{1,3}, 3:{1,2}, 4: {1,5}, 5:{1,4}}

In [ ]:
lp_entropy(G11)

In [ ]:
import scipy
scipy.optimize.show_options('linprog')

In [ ]:
x = np.array([1., 2., 2., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 2., 2., 2., 1.,
       2., 3., 3., 3., 2., 2., 2., 1., 2., 2., 2., 1., 1., 1.])

In [ ]:
x.shape[0]

# Interesting results

In [ ]:
def check_approx_identical_entropy(G1, G2):
    entropy_g1 = lp_entropy(G1)
    entropy_g2 = lp_entropy(G2)
    print(np.round(entropy_g1.x))
    print(np.round(entropy_g2.x))
    return np.array_equal(np.round(entropy_g1.x), np.round(entropy_g2.x))

In [37]:
G1 = {1:{2,3,4,5}, 2:{1,3}, 3:{1,2}, 4: {1,5}, 5:{1,4}}
draw_graph(G1)

In [38]:
G2 = {1:{2,3,4,5}, 2:{1,3,4}, 3:{1,2}, 4: {1,2,5}, 5:{1,4}}
draw_graph(G2)

In [39]:
G3 = {1:{2,3,4,5}, 2:{1,3,4}, 3:{1,2,4}, 4: {1,2,3,5}, 5:{1,4}}
draw_graph(G3)

In [40]:
G4 = {1:{2,3,4,5}, 2:{1,3,4}, 3:{1,2,5}, 4: {1,2,5}, 5:{1,3,4}}
draw_graph(G4)

In [41]:
G5 = {1:{2,4,5}, 2:{1,3,4}, 3:{2,5}, 4: {1,2,5}, 5:{1,3,4}}
draw_graph(G5)

In [42]:
G6 = {1:{2,3,5}, 2:{1,3,4}, 3:{1,2,5}, 4: {2,5}, 5:{1,3,4}}
draw_graph(G6)

In [ ]:
check_approx_identical_entropy(G5,G4)